In [1]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [4]:
input_size = 784
hidden_size = 100
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [5]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, 
                                  transform=torchvision.transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, 
                                  transform=torchvision.transforms.ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size , shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size , shuffle=False)

examples = iter(train_loader)
samples , label = next(examples)
print(samples.shape, label.shape)

torch.Size([100, 1, 28, 28]) torch.Size([100])


In [6]:
class NeuralNet(nn.ModuleList):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()   
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes)

In [7]:
model

NeuralNet(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)

In [30]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [31]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1}, loss = {loss.item()}')

    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for images, labels in test_loader:
            images = images.reshape(-1, 28*28)
            labels = labels.to(device)
            outputs = model(images)

            _, predictions = torch.max(outputs, 1)
            n_samples +=labels.shape[0]
            n_correct +=(predictions == labels).sum().item()
        acc = 100.0 * (n_correct / n_samples)
        print(f'accuracy = {acc}')
        

epoch 1 / 2, step 100, loss = 0.4497401714324951
epoch 1 / 2, step 200, loss = 0.25645020604133606
epoch 1 / 2, step 300, loss = 0.23061120510101318
epoch 1 / 2, step 400, loss = 0.2243112325668335
epoch 1 / 2, step 500, loss = 0.34116727113723755
epoch 1 / 2, step 600, loss = 0.273281455039978
accuracy = 93.83
epoch 2 / 2, step 100, loss = 0.2669540345668793
epoch 2 / 2, step 200, loss = 0.15606042742729187
epoch 2 / 2, step 300, loss = 0.13643260300159454
epoch 2 / 2, step 400, loss = 0.22970733046531677
epoch 2 / 2, step 500, loss = 0.09164124727249146
epoch 2 / 2, step 600, loss = 0.1709711253643036
accuracy = 95.19999999999999
